In [ ]:
import math

import os

import rasterio
import rasterio.plot
from rasterio.plot import plotting_extent
from rasterio.mask import mask 

from osgeo import gdal, osr
gdal.UseExceptions()
    
import shapely
import geopandas as gpd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep

from shutil import copyfile
from PIL import Image

### Abrir arquivos

In [ ]:
# Abrir arquivo de copas
copas = gpd.read_file('../VECTOR/copas_final.gpkg').set_index('id').sort_index()

img = gdal.Open('../IMGS/odm_orthophoto.tif')

### Cortar ortomosaico em patches

In [ ]:
##############################
## Corte com gdal.Translate ##
## ------------------------ ##
##     Sem grade prévia     ##
##############################

# Acessar EPSG do mosaico
prj = img.GetProjection()
srs = osr.SpatialReference(wkt=prj)
epsg = 'EPSG:' + srs.GetAuthorityCode('projcs')

# Tamanho das imagens (px)
patch_size = 256

# Start do ID e do dicionario de poligonos formadores da grade
i = 1
grid_pol = {'id':[], 'geometry':[]}

# Iterar em cada linha da imagem com passos de mesmo tamanho dos patches
for px_x in range(0, img.RasterXSize, patch_size):
    
    # Iterar em cada coluna da imagem com passos de mesmo tamanho dos patches
    for px_y in range(0, img.RasterYSize, patch_size):
        
        # Gerar o dataset do patch na memoria
        ds = gdal.Translate('/vsimem/corte_{i}.tif'.format(i=i), 
                       img, 
                       srcWin = [px_x, px_y, patch_size, patch_size])
        
        # Gerar poligono envolvente da imagem
        xmin, xpixel, _, ymax, _, ypixel = ds.GetGeoTransform()
        width, height = ds.RasterXSize, ds.RasterYSize
        xmax = xmin + width * xpixel
        ymin = ymax + height * ypixel
        geom = shapely.geometry.box(xmin, ymin, xmax, ymax)
        
        
        # Avaliar se o patch corresponde a uma região da imagem com valores válidos (Não zero) e
        # se o patch apresenta alguma copa no seu interior
        if len(ds.ReadAsArray().nonzero()[0]) != 0 and copas.intersects(geom).any():
            
            # Salvar o pacth em disco          
            gdal.Translate('../IMGS/CORTES/corte_{i}.tif'.format(i=i), 
                       img, 
                       srcWin = [px_x, px_y, patch_size, patch_size])
            
            # Inserir poligono na lista
            grid_pol['id'].append(i)
            grid_pol['geometry'].append(geom)
            
            # Atualizar identificador
            i += 1

# Gerar o GeoDataFrame com a grade e salvá-la            
grid = gpd.GeoDataFrame(grid_pol, crs=epsg).set_index('id')
grid.to_file('../VECTOR/grid_from_img.geojson', driver='GeoJSON')

### Cortar vetor das copas

In [ ]:
list_empty = []

for i in grid.index:
    corte = gpd.clip(copas, grid.loc[[i]])
    
    if corte.empty:
        list_empty.append(i)
    else:
        corte.to_file('../VECTOR/CORTES/corte_{}.geojson'.format(i), driver="GeoJSON")
        
print('Grids não salvas: ', list_empty)

### Avaliar se existe alguma copa cortada que foi dividida em duas quando particionada (multipartes)

A partir desse ponto é necessária a existência de um arquivo (grid_selected) correspondente à grade onde será feita a avaliação e com a designação das amostras de treinamento, validação e teste 

Necessário uma coluna "layer" com os valores:
 - grid_from_img-train;
 - grid_from_img-val 
 - grid_from_img-test)

In [ ]:
grid_selected = gpd.read_file('../VECTOR/grid_from_img-all.geojson').set_index('id')

multi = {}
for z in grid_selected.index:
    
    img = rasterio.open('../IMGS/CORTES/corte_{}.tif'.format(z))
    corte = gpd.read_file('../VECTOR/CORTES/corte_{}.geojson'.format(z))
    
    corte.insert(len(corte.columns),'geometry_image', 0)
    corte['geometry_image'] = corte['geometry_image'].astype('object')

    features = []
    for i in range(corte.shape[0]):
        
        if corte.loc[i,'geometry'].geom_type == 'MultiPolygon':
            features.append(i)
    
    if len(features) != 0:
        multi[z] = features

multi

### Inserir coluna de coordenadas das copas em posição X Y da imagem

In [ ]:
for z in grid_selected.index:
    
    img = rasterio.open('../IMGS/CORTES/corte_{}.tif'.format(z))
    corte = gpd.read_file('../VECTOR/CORTES/corte_{}.geojson'.format(z))
    
    corte.insert(len(corte.columns)-1,'geometry_image', 0)
    corte['geometry_image'] = corte['geometry_image'].astype('string')

    for i in corte.index:
        
        # [FEITO] Preciso avaliar a ocorrencia de multipartes 
        # A função .coords e .exterior não se aplicam a multipartes [FEITO]
        coord = list(corte.loc[i,'geometry'].exterior.coords)
        
        segmentation = ''
        # Atenção aqui!! A visualização das annotations indicou que na lista de coors_img, o y precede o x
        for xy in coord:
            segmentation += str(round(img.index(xy[0],xy[1], float)[1],2)) + ', ' + str(round(img.index(xy[0],xy[1], float)[0],2)) + ', '
        
        
        segmentation = segmentation[:-2]
        corte.at[i,'geometry_image'] = segmentation
    
    corte.to_file('../VECTOR/CORTES/EDITADOS/corte_{}.geojson'.format(z), driver="GeoJSON")

### Transferência e conversão dos patches em .tif para a pasta da RmaskCNN em .tif e .jpg

In [ ]:
id_train = grid_selected[grid_selected['layer'] == 'grid_from_img-train'].index
id_val = grid_selected[grid_selected['layer'] == 'grid_from_img-val'].index
id_test = grid_selected[grid_selected['layer'] == 'grid_from_img-test'].index

In [ ]:
for i in id_train:
    copyfile('../IMGS/CORTES/corte_{}.tif'.format(i), '../Mask_RCNN/datasets/canopy/train/images/TIF/{:04}.tif'.format(i))
    
for i in id_val:
    copyfile('../IMGS/CORTES/corte_{}.tif'.format(i), '../Mask_RCNN/datasets/canopy/val/images/TIF/{:04}.tif'.format(i))
    
for i in id_test:
    copyfile('../IMGS/CORTES/corte_{}.tif'.format(i), '../Mask_RCNN/datasets/canopy/real_test/images/TIF/{:04}.tif'.format(i))

In [ ]:
for root, dirs, files in os.walk('../Mask_RCNN/datasets/canopy/', topdown=False):
    
    if root[-4:] == '/TIF':
        
        for name in files:
            
            print(os.path.join(root, name))

            if os.path.splitext(os.path.join(root, name))[1].lower() == ".tif":

                outpath = os.path.split(root)[0]

                if os.path.isfile(os.path.splitext(os.path.join(outpath, name))[0] + ".jpg"):
                    print('A jpeg file already exists for {}'.format(name))

                # If a jpeg is *NOT* present, create one from the tiff.
                else:
                    outfile = os.path.splitext(os.path.join(outpath, name))[0] + ".jpg"
                    try:
                        im = Image.open(os.path.join(root, name))
                        print("Generating jpeg for {}".format(name))
                        im.thumbnail(im.size)
                        im.convert('RGB').save(outfile, "JPEG", quality=100)
                    except Exception as e:
                        print(e)

### Plotar

Plotar 25 amostras aleatórias dos patches com as delimitações das copas

In [ ]:
fig, ax = plt.subplots(5, 5, figsize=(15,15))

for i, axs in zip(grid_selected.sample(25).index, ax.flat):
    
    img = rasterio.open('../IMGS/CORTES/corte_{}.tif'.format(i))
    vetor = gpd.read_file('../VECTOR/CORTES/corte_{}.geojson'.format(i))

    plot_extent = plotting_extent(img)
    ep.plot_rgb(img.read(),
               rgb = [0, 1, 2],
               extent = plot_extent,
               ax=axs)

    vetor.plot(ax=axs, facecolor='none', edgecolor='red')
    
    axs.set_title('Tile {}'.format(i))